In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm

In [2]:
# дата, когда был выгружен файл с последовательностью ТС
str_date = '20200825_1508'

In [3]:
# чтение файла с последовательностью ТС на маршруте (на каких автобусах доехать от А до В)
df_rt_mt_all = pd.read_csv('../new_mt_time/res/df_rt_seq_{}.csv'.format(str_date), sep=';', 
                           encoding='utf-8')

In [4]:
# чтение данных с длиной маршрута, типом (внутри или вне города), данные из Яндекса и СТИ
df_length = pd.read_csv('./data/yandex_sti_w_price.csv', sep=';', encoding='utf-8')

In [5]:
df_length.head(3)

,line_id,route_number,type_ts,route_long_name,реестр_номер,реестр_имя,тип_тс,city,length,price,region
0,1704841444,1,bus,72-й квартал_Станция Новокуйбышевская,1,72-й квартал - Станция Новокуйбышевская,автобус,Новокуйбышевск,10.53,25,муниципальный
1,3245401927,1,bus,Автостанция Нефтегорск_Дмитриевка,1,г. Нефтегорск - с. Дмитриевка,автобус,Нефтегорский район,57.17,-,муниципальный
2,1704841008,1,bus,Автостанция_Автостанция,None,None,NaN,Похвистнево,19.89,-,муниципальный


In [6]:
np_dfl = df_length.to_numpy()
ind_tp = list(df_length.columns).index('type_ts')
ind_reg = list(df_length.columns).index('region')
ind_len = list(df_length.columns).index('length')
ind_prc = list(df_length.columns).index('price')
ind_ld = list(df_length.columns).index('line_id')

In [7]:
# подсчет стоимости минимальной, максимальной и руб/км

lst_min = []
lst_max = []
lst_prc_km = []
i=0
for i in tqdm(range(len(np_dfl))):
    tp_ts = np_dfl[i, ind_tp]
    reg = np_dfl[i, ind_reg]
    lngth = np_dfl[i, ind_len]
    lineid = np_dfl[i, ind_ld]
    
    try:
        prc_ttl = int(np_dfl[i, ind_prc])
    except:
        prc_ttl = np_dfl[i, ind_prc]    
    if tp_ts == 'water':
        mn_min = prc_ttl
        mn_max = prc_ttl
        prc_km = round(int(prc_ttl) / lngth, 1)
    elif tp_ts == 'suburban':
        mn_min = 80
        mn_max = prc_ttl
        prc_km = 2
    else:
        if 'муницип' in reg:
            if prc_ttl == '-':
                prc_ttl = 29
            mn_min = prc_ttl
            mn_max = prc_ttl
            prc_km = round(int(prc_ttl) / lngth, 1)
        else:
            if prc_ttl == '-':
                    prc_ttl = round(lngth * 3, 1)
            if prc_ttl < 60:
                mn_min = prc_ttl
                mn_max = 60
                prc_km = 3             
            else:
                mn_min = 60
                mn_max = round(lngth * 3, 1)
                prc_km = 3
    #
    lst_prc_km.append([lineid, mn_min, mn_max, prc_km])
# 

In [8]:
df_total_price = pd.DataFrame(lst_prc_km, columns=['line_id', 
                                                   'min_price', 'max_price', 'rub_km'])

In [154]:
# df_total_price.to_csv('df_total_price.csv', sep=';', encoding='utf-8-sig', index=False)

In [155]:
# df_total_price = pd.read_csv('./res/df_total_price.csv', sep=';', encoding='utf-8')

In [9]:
df_total_price.head(2)

,line_id,min_price,max_price,rub_km
0,1704841444,25.0,25.0,2.4
1,3245401927,29.0,29.0,0.5


In [10]:
df_total_price = df_total_price.merge(df_length[['line_id', 'type_ts', 'region']], 
                                      how='left', on=['line_id'])
# 

In [11]:
df_rt_mt_all.head(2)

,id_reg_from,id_reg_to,seq_in_rt,line_id,tp_ts,name,distance
0,2,1,1,2193179089,bus,30,1.38
1,2,1,2,2193179361,minibus,261,8.07


In [12]:
df_rt_mt_all = df_rt_mt_all.drop_duplicates(subset=['id_reg_from', 
                                                    'id_reg_to', 'seq_in_rt']).reset_index(drop=True)

In [13]:
# создание numpy.array - так быстрее, чем через pandas
# некоторые id надо заменить, так как они не сходятся, соответствия были найдены вручную
np_mt = df_rt_mt_all.to_numpy()
ind_li = list(df_rt_mt_all.columns).index('line_id')

lst_bad = ['grouped_id_s9606096_s9606549', 'grouped_id_s9606549_s9606096', 
          'grouped_id_s9606629_s9606096', 'grouped_id_s9606096_s9606629',
          'grouped_id_s9606246_s9606374', 'grouped_id_s9606374_s9606246',
          'grouped_id_s9606404_s9606096', 'grouped_id_s9606096_s9606404']
# 
lst_good = ['lh_6318x6317_9606096_g20_4', 'lh_6320x6319_9606549_g20_4', 
          'lh_6324x6323_9606629_g20_4', 'lh_6340x6339_9606096_g20_4',
          'lh_6571x6573_9606246_g20_4', 'lh_6562_9606374_g20_4',
          'lh_6519_9606404_g20_4', 'lh_6528_9606096_g20_4']
# 

In [14]:
lst_id_new = []
i=0
for i in tqdm(range(len(np_mt))):
    if np_mt[i, ind_li] in lst_bad:
        ind_bad = lst_bad.index(np_mt[i, ind_li])
        lst_id_new.append(lst_good[ind_bad])
    else:
        lst_id_new.append(np_mt[i, ind_li])
# 

In [15]:
df_rt_mt_all['line_id'] = lst_id_new

In [16]:
# подтягивание стоимости к последвоательностям
df_mt_n_price = df_rt_mt_all.merge(df_total_price[['line_id', 'min_price',
                                                   'max_price', 'rub_km','region']], 
                                      how='left', on=['line_id'])

In [17]:
df_mt_n_price.head(2)

,id_reg_from,id_reg_to,seq_in_rt,line_id,tp_ts,name,distance,min_price,max_price,rub_km,region
0,2,1,1,2193179089,bus,30,1.38,32.0,32.0,1.9,муниципальный
1,2,1,2,2193179361,minibus,261,8.07,35.0,35.0,1.3,муниципальный


In [18]:
np_mnp = df_mt_n_price.to_numpy()
ind_ifr = list(df_mt_n_price.columns).index('id_reg_from')
ind_ito = list(df_mt_n_price.columns).index('id_reg_to')
ind_tp = list(df_mt_n_price.columns).index('tp_ts')
ind_dist = list(df_mt_n_price.columns).index('distance')
ind_min = list(df_mt_n_price.columns).index('min_price')
ind_max = list(df_mt_n_price.columns).index('max_price')
ind_rkm = list(df_mt_n_price.columns).index('rub_km')
ind_reg = list(df_mt_n_price.columns).index('region')

In [19]:
# расчет стоимости проезда: стоимость км * км
lst_money_per_mt = []
i=0
for i in tqdm(range(len(np_mnp))):
    idfrom = np_mnp[i, ind_ifr]
    idto = np_mnp[i, ind_ito]
    tp_ts = str(np_mnp[i, ind_tp])
    distnc = np_mnp[i, ind_dist]
    mn_min = np_mnp[i, ind_min]
    mn_max = np_mnp[i, ind_max]
    rub_km = np_mnp[i, ind_rkm]
    region = str(np_mnp[i, ind_reg])
    
    if (('муниципальн' in region) | (tp_ts in ['water', 'underground'])):
        mn_per_mt = mn_min
    else:
        mn_per_mt = distnc * rub_km
        if mn_per_mt < mn_min:
            mn_per_mt = mn_min
        if mn_per_mt > mn_max:
            mn_per_mt = mn_max
        #
    lst_money_per_mt.append(mn_per_mt)
# 

In [20]:
df_mt_n_price['mn_per_mt'] = lst_money_per_mt

In [168]:
# df_mt_n_price.to_csv('./res/df_mt_n_price.csv', sep=';', encoding='utf-8-sig', index=False)

In [21]:
# суммирование всех ТС, для получения общей стоимости проезда на всех ТС от А до В
grp_mtnp = pd.DataFrame(df_mt_n_price.groupby(['id_reg_from', 
                                               'id_reg_to']).sum()['mn_per_mt']).reset_index()
grp_mtnp = grp_mtnp.reset_index(drop=True)

In [22]:
grp_mtnp.head(3)

,id_reg_from,id_reg_to,mn_per_mt
0,1,2,99.0
1,1,3,67.0
2,1,4,124.0


In [23]:
# лучше переделать через numpy, так быстрее
# создание списков стоимости и маршуртов от А до В 
# (просто чтобы можно было видеть, как складывалась цена)
lst_cmm_mn = []
lst_cmm_rts = []
i=0
for i in tqdm(range(len(grp_mtnp))):
    id_from = grp_mtnp.id_reg_from[i]
    id_to = grp_mtnp.id_reg_to[i]
    prt_df = df_mt_n_price[(df_mt_n_price.id_reg_from == id_from) 
                           & (df_mt_n_price.id_reg_to == id_to)]
    prt_df['mn_per_mt'] = prt_df['mn_per_mt'].astype(str)
    lst_cmm_mn.append(str(list(prt_df['mn_per_mt'])))
    lst_cmm_rts.append(str(list(prt_df['line_id'])))
# 

C:\Users\ochristm\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [25]:
grp_mtnp['mn_sep'] = lst_cmm_mn
grp_mtnp['rts_sep'] = lst_cmm_rts

In [26]:
grp_mtnp.head(2)

,id_reg_from,id_reg_to,mn_per_mt,mn_sep,rts_sep
0,1,2,99.0,"['32.0', '35.0', '32.0']","['2193179055', '2193179361', '2193179089']"
1,1,3,67.0,"['32.0', '35.0']","['2193179055', '2193179361']"


In [27]:
# чтение файла с геометрией по маршрутам между районами (только авто)
# чтобы подтянуть данные от стоимости к геометрии
gdf_rt_all = gpd.read_file('../data/res/big_ok/matrice_len_time.json', encoding='utf-8')

In [28]:
gdf_rt_all.columns

Index(['index', 'id_reg_from', 'id_reg_to', 'length', 'time_auto_avg',
       'time_auto_traff', 'time_pubtr', 'x_from', 'y_from', 'x_to', 'y_to',
       'mnt_from_reg', 'mnt_to_reg', 'cnt_transprt', 'geometry'],
      dtype='object')

In [29]:
# gdf_rt_all2 = gdf_rt_all[['id_reg_from', 'id_reg_to', 'length', 
#                           'time_auto_avg','geometry']].copy()
gdf_rt_all2 = gdf_rt_all.copy()

In [30]:
len(gdf_rt_all2)

13110

In [31]:
# подтягивание, описано выше
gdf_rt_all3 = gdf_rt_all2.merge(grp_mtnp, how='left', on=['id_reg_from', 'id_reg_to'])

In [32]:
len(gdf_rt_all3)

13110

In [33]:
gdf_rt_all3.head(3)

,index,id_reg_from,id_reg_to,length,time_auto_avg,time_auto_traff,time_pubtr,x_from,y_from,x_to,y_to,mnt_from_reg,mnt_to_reg,cnt_transprt,geometry,mn_per_mt,mn_sep,rts_sep
0,0,1,2,67.75,69.3,69.3,162.0,49.847273,53.484996,50.257003,53.199730,30,25,3,"LINESTRING (49.84730 53.48513, 49.84669 53.485...",99.0,"['32.0', '35.0', '32.0']","['2193179055', '2193179361', '2193179089']"
1,1,2,1,59.54,69.4,69.4,167.9,50.257003,53.199730,49.847273,53.484996,25,30,3,"LINESTRING (50.25715 53.19960, 50.25656 53.199...",99.0,"['32.0', '35.0', '32.0']","['2193179089', '2193179361', '2193179055']"
2,2,2,3,3.13,15.2,15.2,65.5,50.257003,53.199730,50.282329,53.211924,25,30,1,"LINESTRING (50.25685 53.19986, 50.25628 53.199...",35.0,['35.0'],['2193179362']


In [34]:
# 47 = Стоимость бензина, руб. / л
# 0.11 = Расход бензина, л / км	
# 2.12 = Стоимость времени в пути, руб, / мин
# 5 = время "на парковку" - проверить, не было ли добавлено 

In [35]:
# вычисление соимости проезда на авто
# gdf_rt_all3['time_auto_avg'] = gdf_rt_all3['time_auto_avg'] + 5 + 5
# gdf_rt_all3['time_auto_traff'] = gdf_rt_all3['time_auto_traff'] + 5 + 5

gdf_rt_all3['mn_per_auto'] = 47*0.11*gdf_rt_all3['length'] + gdf_rt_all3['time_auto_avg']*2.12

In [36]:
gdf_rt_all3.head(3)

,index,id_reg_from,id_reg_to,length,time_auto_avg,time_auto_traff,time_pubtr,x_from,y_from,x_to,y_to,mnt_from_reg,mnt_to_reg,cnt_transprt,geometry,mn_per_mt,mn_sep,rts_sep,mn_per_auto
0,0,1,2,67.75,69.3,69.3,162.0,49.847273,53.484996,50.257003,53.199730,30,25,3,"LINESTRING (49.84730 53.48513, 49.84669 53.485...",99.0,"['32.0', '35.0', '32.0']","['2193179055', '2193179361', '2193179089']",497.1835
1,1,2,1,59.54,69.4,69.4,167.9,50.257003,53.199730,49.847273,53.484996,25,30,3,"LINESTRING (50.25715 53.19960, 50.25656 53.199...",99.0,"['32.0', '35.0', '32.0']","['2193179089', '2193179361', '2193179055']",454.9498
2,2,2,3,3.13,15.2,15.2,65.5,50.257003,53.199730,50.282329,53.211924,25,30,1,"LINESTRING (50.25685 53.19986, 50.25628 53.199...",35.0,['35.0'],['2193179362'],48.4061


In [37]:
# gdf_rt_all3['time_pubtr'] = gdf_rt_all3['time_pubtr'].fillna(gdf_rt_all3['time_auto_avg'])

In [38]:
# прибавление стоимости времени в пути
gdf_rt_all3['mn_per_mt'] = gdf_rt_all3['mn_per_mt'] + gdf_rt_all3['time_pubtr'] * 2.12

In [39]:

# gdf_rt_all3['mn_per_mt'] = gdf_rt_all3['mn_per_mt'].fillna(gdf_rt_all3['time_pubtr'] * 2.12)

In [40]:
gdf_rt_all3.head(3)

,index,id_reg_from,id_reg_to,length,time_auto_avg,time_auto_traff,time_pubtr,x_from,y_from,x_to,y_to,mnt_from_reg,mnt_to_reg,cnt_transprt,geometry,mn_per_mt,mn_sep,rts_sep,mn_per_auto
0,0,1,2,67.75,69.3,69.3,162.0,49.847273,53.484996,50.257003,53.199730,30,25,3,"LINESTRING (49.84730 53.48513, 49.84669 53.485...",442.440,"['32.0', '35.0', '32.0']","['2193179055', '2193179361', '2193179089']",497.1835
1,1,2,1,59.54,69.4,69.4,167.9,50.257003,53.199730,49.847273,53.484996,25,30,3,"LINESTRING (50.25715 53.19960, 50.25656 53.199...",454.948,"['32.0', '35.0', '32.0']","['2193179089', '2193179361', '2193179055']",454.9498
2,2,2,3,3.13,15.2,15.2,65.5,50.257003,53.199730,50.282329,53.211924,25,30,1,"LINESTRING (50.25685 53.19986, 50.25628 53.199...",173.860,['35.0'],['2193179362'],48.4061


In [42]:
# там, где нет последовательности маршрутов - Яндекс построил пешком,
# изменить кол-во ТС на 0 и посчитать стоимость времени в пути

lst_new_money_mt=[]
lst_new_cnt_transprt=[]
i=0
for i in tqdm(range(len(gdf_rt_all3))):
    # if is nan
    if gdf_rt_all3.mn_sep[i] != gdf_rt_all3.mn_sep[i]:
        money = gdf_rt_all3.time_pubtr[i] * 2.12
        cnt_transprt = 0
    else:
        money = gdf_rt_all3.mn_per_mt[i]
        cnt_transprt = gdf_rt_all3.cnt_transprt[i]
    #
    lst_new_money_mt.append(money)
    lst_new_cnt_transprt.append(cnt_transprt)
#

In [43]:
gdf_rt_all3['cnt_transprt'] = lst_new_cnt_transprt
gdf_rt_all3['mn_per_mt'] = lst_new_money_mt

In [46]:
gdf_rt_all3 = gdf_rt_all3.rename(columns={'length':'dist_auto_km', 
                                          'mn_per_auto':'total_money_auto', 
                                          'mn_per_mt':'total_money_pubtr', 
                                          'mn_sep':'tariffs_sep', 
                                          'rts_sep' :'routes_sep'})

In [47]:
# это просто для изменения порядка столбцов
gdf_rt_all3 = gdf_rt_all3[['id_reg_from', 'id_reg_to', 'dist_auto_km', 
                           'time_auto_avg','time_auto_traff', 'time_pubtr',
                           'total_money_auto', 'total_money_pubtr', 
                           'cnt_transprt', 'tariffs_sep', 'routes_sep',
                           'mnt_from_reg', 'mnt_to_reg',
                           'x_from', 'y_from', 'x_to', 'y_to', 
                           'geometry']]

In [48]:
gdf_rt_all4 = gdf_rt_all3.copy()
del gdf_rt_all4['geometry']
gdf_rt_all4.to_csv('./res/gdf_rt_time_price_csv.csv', sep=';', encoding='utf-8-sig', index=False)

In [49]:
gdf_rt_all3.to_file('./res/gdf_rt_time_price.json', driver='GeoJSON', encoding='utf-8')